In [ ]:
import sys
sys.path.append('../implementation')
import numpy as np
import pandas as pd
import ast
from tqdm import tqdm
from util import lognormpdf
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import scipy.stats as stats
import scikit_posthocs as sci_post
import time
from statistics import mean
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load results
boardrooms_results_hmm = pd.read_pickle('../output/boardrooms/boardrooms_hmm.pkl')
boardrooms_results_cm = pd.read_pickle('../output/boardrooms/boardrooms_cm.pkl')
boardrooms_results_wall = pd.read_pickle('../output/boardrooms/boardrooms_wall.pkl')
boardrooms_results_knn = pd.read_pickle('../output/boardrooms/boardrooms_knn.pkl')
boardrooms_results_ac = pd.read_pickle('../output/boardrooms/boardrooms_ac.pkl')
boardrooms_results_af = pd.read_pickle('../output/boardrooms/boardrooms_af.pkl')
boardrooms_results_ada = pd.read_pickle('../output/boardrooms/boardrooms_ada_nb.pkl')

ks = [1, 5, 10, 20, 50, 100]

In [ ]:
avg_rank = pd.DataFrame()

t = 1
for i, row in boardrooms_results_hmm[boardrooms_results_hmm.task == t].iterrows():
    try:
        average_rank = mean(row['rank'])
    except: 
        average_rank = None
    temp_df = pd.DataFrame({'alg': ['hmm'], 'task': [t], 'average_rank': [average_rank]})
    avg_rank = pd.concat([avg_rank, temp_df], ignore_index=True)
for i, row in boardrooms_results_cm[boardrooms_results_cm.task == t].iterrows():
    try:
        average_rank = mean(row['rank'])
    except: 
        average_rank = None
    temp_df = pd.DataFrame({'alg': ['cm'], 'task': [t], 'average_rank': [average_rank]})
    avg_rank = pd.concat([avg_rank, temp_df], ignore_index=True)
for i, row in boardrooms_results_af[boardrooms_results_af.task == t].iterrows():
    try:
        average_rank = mean(row['rank'])
    except: 
        average_rank = None
    temp_df = pd.DataFrame({'alg': ['af'], 'task': [t], 'average_rank': [average_rank]})
    avg_rank = pd.concat([avg_rank, temp_df], ignore_index=True)
for i, row in boardrooms_results_knn[boardrooms_results_knn.task == t].iterrows():
    try:
        average_rank = mean(row['rank'])
    except: 
        average_rank = None
    temp_df = pd.DataFrame({'alg': ['knn'], 'task': [t], 'average_rank': [average_rank]})
    avg_rank = pd.concat([avg_rank, temp_df], ignore_index=True)
for i, row in boardrooms_results_ada[boardrooms_results_ada.task == t].iterrows():
    try:
        average_rank = mean(row['rank'])
    except: 
        average_rank = None
    temp_df = pd.DataFrame({'alg': ['ada_nb'], 'task': [t], 'average_rank': [average_rank]})
    avg_rank = pd.concat([avg_rank, temp_df], ignore_index=True)
    
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
def run_comp(data, task, alpha=0.05):
    posthoc_dun = None
    avg_rank_data = data[data['task'] == task]
    hmm_rank_val = avg_rank_data[avg_rank_data['alg'] == 'hmm']['average_rank'].values
    cm_rank_val = avg_rank_data[avg_rank_data['alg'] == 'cm']['average_rank'].values
    af_rank_val = avg_rank_data[avg_rank_data['alg'] == 'af']['average_rank'].values
    knn_rank_val = avg_rank_data[avg_rank_data['alg'] == 'knn']['average_rank'].values
    ada_nb_rank_val = avg_rank_data[avg_rank_data['alg'] == 'ada_nb']['average_rank'].values
    
    hmm_p = stats.shapiro(hmm_rank_val).pvalue
    cm_p = stats.shapiro(cm_rank_val).pvalue
    af_p = stats.shapiro(af_rank_val).pvalue
    knn_p = stats.shapiro(knn_rank_val).pvalue
    ada_nb_p = stats.shapiro(ada_nb_rank_val).pvalue
    norm_p = np.array([hmm_p, cm_p, af_p, knn_p, ada_nb_p])
    
    if all(p >= alpha for p in norm_p):
        print('ANOVA')
        statistic, pvalue = stats.f_oneway(hmm_rank_val, cm_rank_val, af_rank_val, knn_rank_val, ada_nb_rank_val)
    else:
        print('Kruskal')
        kruskal_res = stats.kruskal(hmm_rank_val, cm_rank_val, af_rank_val, knn_rank_val, ada_nb_rank_val)
        statistic = kruskal_res.statistic
        pvalue = kruskal_res.pvalue
        if pvalue <= alpha:
            posthoc_dun = sci_post.posthoc_dunn(avg_rank_data, val_col='average_rank',
                                          group_col = 'alg', p_adjust = 'bonferroni')
    return statistic, pvalue, posthoc_dun

statistic, p_val, posthoc_dun = run_comp(avg_rank, t)
print(f'statistic: {statistic}, p_val: {p_val}\nmult_comp:\n {posthoc_dun}\n')
plt_data = avg_rank[avg_rank['task'] == t].dropna()
sns.catplot(
    data=plt_data, x='average_rank', y='alg', kind='box', height=5
).set(title='Average Rank Per Session for Boardrooms')

In [ ]:
fig = plt.figure(figsize=(2*6.4, 4.8))
plt.rcParams.update({'axes.titlesize': 15, 'axes.labelsize': 15, 'xtick.labelsize':12, 'xtick.labelsize':12})
ax = fig.add_axes([0,0,1,1])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.set(xlabel='Interactions Observed', ylabel='Avg. Rank')
ax.set_ylim((0, 500))
#     ax.set_xlim((2, 12))

t = 1
rank_over_time_hmm = pd.DataFrame()
rank_over_time_cm = pd.DataFrame()
rank_over_time_af = pd.DataFrame()
rank_over_time_knn = pd.DataFrame()
rank_over_time_ada_nb = pd.DataFrame()
for i, row in boardrooms_results_hmm[boardrooms_results_hmm.task == t].iterrows():
    temp_df = pd.DataFrame()
    temp_df[row['participant_id']] = row['rank']
    rank_over_time_hmm = pd.concat([rank_over_time_hmm, temp_df], axis=1, ignore_index=True)
for i, row in boardrooms_results_cm[boardrooms_results_cm.task == t].iterrows():
    temp_df = pd.DataFrame()
    temp_df[row['participant_id']] = row['rank']
    rank_over_time_cm = pd.concat([rank_over_time_cm, temp_df], axis=1, ignore_index=True)
for i, row in boardrooms_results_af[boardrooms_results_af.task == t].iterrows():
    temp_df = pd.DataFrame()
    temp_df[row['participant_id']] = row['rank']
    rank_over_time_af = pd.concat([rank_over_time_af, temp_df], axis=1, ignore_index=True)
for i, row in boardrooms_results_knn[boardrooms_results_knn.task == t].iterrows():
    temp_df = pd.DataFrame()
    temp_df[row['participant_id']] = row['rank']
    rank_over_time_knn = pd.concat([rank_over_time_knn, temp_df], axis=1, ignore_index=True)
for i, row in boardrooms_results_ada[boardrooms_results_ada.task == t].iterrows():
    temp_df = pd.DataFrame()
    temp_df[row['participant_id']] = row['rank']
    rank_over_time_ada_nb = pd.concat([rank_over_time_ada_nb, temp_df], axis=1, ignore_index=True)

sems_hmm = rank_over_time_hmm.std(axis=1) / np.sqrt(rank_over_time_hmm.count(axis=1))
mean_hmm = rank_over_time_hmm.mean(axis=1)

sems_cm = rank_over_time_cm.std(axis=1) / np.sqrt(rank_over_time_cm.count(axis=1))
mean_cm = rank_over_time_cm.mean(axis=1)

sems_af = rank_over_time_af.std(axis=1) / np.sqrt(rank_over_time_af.count(axis=1))
mean_af = rank_over_time_af.mean(axis=1)

sems_knn = rank_over_time_knn.std(axis=1) / np.sqrt(rank_over_time_knn.count(axis=1))
mean_knn = rank_over_time_knn.mean(axis=1)

sems_ada_nb = rank_over_time_ada_nb.std(axis=1) / np.sqrt(rank_over_time_ada_nb.count(axis=1))
mean_ada_nb = rank_over_time_ada_nb.mean(axis=1)

mean_hmm.plot(title=f'Aggregate Prediction Rank for Boardrooms', label='HMM', color='#1F77B4')
mean_cm.plot(title=f'Aggregate Prediction Rank for Boardrooms', label='Competing Models', color='#d95f02')
mean_af.plot(title=f'Aggregate Prediction Rank for Boardrooms', label='Analytic Focus', color='#548B54')
mean_knn.plot(title=f'Aggregate Prediction Rank for Boardrooms', label='KNN', color='#DC143C')
mean_ada_nb.plot(title=f'Aggregate Prediction Rank for Boardrooms', label='AdaBoost-NB', color='#8A2BE2')

#     axs[ai].fill_between(list(range(len(mean_hmm))), mean_hmm-2*sems_hmm,mean_hmm+2*sems_hmm, color='#1F77B4', alpha=0.3, zorder=100)
#     axs[ai].fill_between(list(range(len(mean_cm))), mean_cm-2*sems_cm,mean_cm+2*sems_cm, color='#d95f02', alpha=0.3, zorder=100)
#     axs[ai].fill_between(list(range(len(mean_af))), mean_af-2*sems_af,mean_af+2*sems_af, color='#548B54', alpha=0.3, zorder=100)
#     axs[ai].fill_between(list(range(len(mean_knn))), mean_knn-2*sems_knn,mean_knn+2*sems_knn, color='#DC143C', alpha=0.3, zorder=100)
#     axs[ai].fill_between(list(range(len(mean_ada_nb))), mean_ada_nb-2*sems_ada_nb,mean_ada_nb+2*sems_ada_nb, color='#8A2BE2', alpha=0.3, zorder=100)        

ax.legend()
plt.show()

In [ ]:
# Comparing next click prediction with Competing Models and HMM
# fig, axs = plt.subplots(1, 2, sharey=True, figsize=(2*6.4, 4.8))
fig = plt.figure(figsize=(2*6.4, 4.8))
plt.rcParams.update({'axes.titlesize': 15, 'axes.labelsize': 15, 'xtick.labelsize':12, 'xtick.labelsize':12})
ax = fig.add_axes([0,0,1,1])
ax.set(ylabel='Avg. Accuracy')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.set(xlabel='k')
ax.set_ylim((0, 1.05))

    
labels = ['ncp-1','ncp-5','ncp-10','ncp-20','ncp-50','ncp-100']
x = np.arange(len(labels))  # the label locations
width = 0.15  # the width of the bars
t = 1
df_temp_hmm = boardrooms_results_hmm[boardrooms_results_hmm.task == t][[f'ncp-{k}' for k in ks]]
err_hmm = df_temp_hmm.std() / np.sqrt(len(df_temp_hmm))
df_temp_cm = boardrooms_results_cm[boardrooms_results_cm.task == t][[f'ncp-{k}' for k in ks]]
err_cm = df_temp_cm.std() / np.sqrt(len(df_temp_cm))
df_temp_knn = boardrooms_results_knn[boardrooms_results_knn.task == t][[f'ncp-{k}' for k in ks]]
err_knn = df_temp_knn.std() / np.sqrt(len(df_temp_knn))
df_temp_af = boardrooms_results_af[boardrooms_results_af.task == t][[f'ncp-{k}' for k in ks]]
err_af = df_temp_af.std() / np.sqrt(len(df_temp_af))
df_temp_ada = boardrooms_results_ada[boardrooms_results_ada.task == t][[f'ncp-{k}' for k in ks]]
err_ada = df_temp_ada.std() / np.sqrt(len(df_temp_ada))

hmm_results = df_temp_hmm.mean()
cm_results = df_temp_cm.mean()
knn_results = df_temp_knn.mean()
af_results = df_temp_af.mean()
ada_results = df_temp_ada.mean()

hmm_bar = ax.bar(x - width*2, hmm_results, width, label='HMM', yerr=err_hmm)
cm_bar = ax.bar(x - width, cm_results, width, label='Competing Models', yerr=err_cm, color='#d95f02', alpha=0.6)
knn_bar = ax.bar(x, knn_results, width, label='KNN', yerr=err_knn, color='#DC143C', alpha=0.6)
af_bar = ax.bar(x + width, af_results, width, label='Analytic Focus', yerr=err_af, color='#548B54', alpha=0.6)
ada_bar = ax.bar(x + width*2, ada_results, width, label='AdaBoost-NB', yerr=err_ada, color='#8A2BE2', alpha=0.6)

ax.set_title('Aggregate Next Click Prediction for Boardrooms')
ax.set_xticks(x, labels)
ax.legend()
plt.show()

In [ ]:
# Comparing bias with Wall, Competing Models, HMM, and AC
fig, axs = plt.subplots(4, 2, sharey=True, figsize=(20, 15))
fig.tight_layout(pad=6)
plt.rcParams.update({'axes.titlesize': 15, 'axes.labelsize': 15, 'xtick.labelsize':12, 'xtick.labelsize':12})
fig.suptitle('Aggregate Bias Detection for Boardrooms Data', fontsize=20)
fig.text(0.5, 0.03, 'Interactions Observed', ha='center')
fig.text(0.03, 0.5, 'Avg. Bias', va='center', rotation='vertical')

for (m,n), ax in np.ndenumerate(axs):
    ax.set_ylim((0, 1.05))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')

columns = ['age', 'female', 'industry', 'medianpay', 'mktcap', 'tenure', 'unrelated']

bias_metric_per_task_hmm = {'age': 'bias-bias_age', 'female': 'bias-bias_female',
                           'industry': 'bias-bias_industry', 'medianpay': 'bias-bias_medianpay',
                           'mktcap': 'bias-bias_mktcap', 'tenure': 'bias-bias_tenure', 'unrelated':'bias-bias_unrelated'}
bias_metric_per_task_cm = {'age': 'bias-age', 'female': 'bias-female',
                           'industry': 'bias-industry', 'medianpay': 'bias-medianpay',
                           'mktcap': 'bias-mktcap', 'tenure': 'bias-tenure', 'unrelated':'bias-unrelated'}
bias_metric_per_task_wall = {'age': 'bias-age', 'female': 'bias-female',
                           'industry': 'bias-industry', 'medianpay': 'bias-medianpay',
                           'mktcap': 'bias-mktcap', 'tenure': 'bias-tenure', 'unrelated':'bias-unrelated'}
bias_metric_per_task_ac = {'age': 'bias-age_disc', 'female': 'bias-female_disc',
                           'industry': 'bias-industry', 'medianpay': 'bias-medianpay_disc',
                           'mktcap': 'bias-mktcap_disc', 'tenure': 'bias-tenure_disc', 'unrelated':'bias-unrelated_disc'}

t = 1
index_1 = 0
index_2 = 0
for ai, attr in enumerate(columns):
    bias_over_time_hmm = pd.DataFrame()
    bias_over_time_cm = pd.DataFrame()
    bias_over_time_wall = pd.DataFrame()
    bias_over_time_knn = pd.DataFrame()
    bias_over_time_ac = pd.DataFrame()
    for i, row in boardrooms_results_ac[boardrooms_results_ac.task == t].iterrows():
        temp_df = pd.DataFrame()
        temp_df[row['participant_id']] = row[bias_metric_per_task_ac[attr]]
        bias_over_time_ac = pd.concat([bias_over_time_ac, temp_df], axis=1, ignore_index=True)
    for i, row in boardrooms_results_hmm[boardrooms_results_hmm.task == t].iterrows():
        temp_df = pd.DataFrame()
        temp_df[row['participant_id']] = row[bias_metric_per_task_hmm[attr]]
        bias_over_time_hmm = pd.concat([bias_over_time_hmm, temp_df], axis=1, ignore_index=True)
    for i, row in boardrooms_results_cm[boardrooms_results_cm.task == t].iterrows():
        temp_df = pd.DataFrame()
        temp_df[row['participant_id']] = row[bias_metric_per_task_cm[attr]]
        bias_over_time_cm = pd.concat([bias_over_time_cm, temp_df], axis=1, ignore_index=True)
    for i, row in boardrooms_results_wall[boardrooms_results_wall.task == t].iterrows():
        temp_df = pd.DataFrame()
        temp_df[row['participant_id']] = row[bias_metric_per_task_wall[attr]]
        bias_over_time_wall = pd.concat([bias_over_time_wall, temp_df], axis=1, ignore_index=True)
        
    sems_hmm = bias_over_time_hmm.std(axis=1) / np.sqrt(bias_over_time_hmm.count(axis=1))
    mean_hmm = bias_over_time_hmm.mean(axis=1)
    
    sems_cm = bias_over_time_cm.std(axis=1) / np.sqrt(bias_over_time_cm.count(axis=1))
    mean_cm = bias_over_time_cm.mean(axis=1)
    
    sems_wall = bias_over_time_wall.std(axis=1) / np.sqrt(bias_over_time_wall.count(axis=1))
    mean_wall = bias_over_time_wall.mean(axis=1)
    
    sems_knn = bias_over_time_knn.std(axis=1) / np.sqrt(bias_over_time_knn.count(axis=1))
    mean_knn = bias_over_time_knn.mean(axis=1)
    
    sems_ac = bias_over_time_ac.std(axis=1) / np.sqrt(bias_over_time_ac.count(axis=1))
    mean_ac = bias_over_time_ac.mean(axis=1)
    
    mean_hmm.plot(ax=axs[index_1, index_2], title=f'Aggregate Bias Detection for {attr}', label='HMM', color='#1F77B4', legend = 0)
    mean_cm.plot(ax=axs[index_1, index_2], title=f'Aggregate Bias Detection for {attr}', label='Competing Models', color='#d95f02', legend = 0)
    mean_wall.plot(ax=axs[index_1, index_2], title=f'Aggregate Bias Detection for {attr}', label='Wall', color='#8FBC8F', legend = 0)
    mean_ac.plot(ax=axs[index_1, index_2], title=f'Aggregate Bias Detection for {attr}', label='AC', color='#8A2BE2', legend = 0)
    
    axs[index_1, index_2].fill_between(list(range(len(mean_hmm))), mean_hmm-2*sems_hmm,mean_hmm+2*sems_hmm, color='#1F77B4', alpha=0.3, zorder=100)
    axs[index_1, index_2].fill_between(list(range(len(mean_cm))), mean_cm-2*sems_cm,mean_cm+2*sems_cm, color='#d95f02', alpha=0.3, zorder=100)
    axs[index_1, index_2].fill_between(list(range(len(mean_wall))), mean_wall-2*sems_wall,mean_wall+2*sems_wall, color='#8FBC8F', alpha=0.3, zorder=100)
    axs[index_1, index_2].fill_between(list(range(len(mean_ac))), mean_ac-2*sems_ac,mean_ac+2*sems_ac, color='#8A2BE2', alpha=0.3, zorder=100)

    if ai != 6:
        if (index_2 == 1):
            index_2 = 0
            index_1 += 1
        else:
            index_2 += 1
    else:
        handles, labels = axs[index_1, index_2].get_legend_handles_labels()
        fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.07))
fig.delaxes(axs[3][1])